In [1]:
from ortools.sat.python import cp_model

In [4]:
N = 0
M = 0
classes = []
room_capacities = []

def read_input(file_name):
    global N, M, classes, room_capacities

    with open(file_name) as file:
        lines = [line.rstrip() for line in file]

    N, M = tuple(int(x) for x in lines[0].split())

    for i in range(N):
        classes.append(tuple(int(x) for x in lines[i + 1].split()))

    room_capacities = [int(x) for x in lines[-1].split()]

In [5]:
read_input('./data/input0.txt')

# MODEL

In [6]:
model = cp_model.CpModel()

In [17]:
# VARIABLES & DOMAINS
SP = []
SP_isValid = []

for i in range(N):
    SP.append(model.NewIntVar(-1, 59, f'SP[{i}]'))
    SP_isValid.append(model.NewBoolVar(f'SP_isValid[{i}]'))


R = []
R_isValid = []

for i in range(N):
    R.append(model.NewIntVar(-1, M, f'R[{i}]'))
    R_isValid.append(model.NewBoolVar(f'R_isValid[{i}]'))


In [18]:
# CONSTRAINTS

for i in range(N):
    model.Add(
        (SP[i] == -1) == (R[i] == -1)
    )

    model.AddLinearExpressionInDomain(
        SP[i], 
        cp_model.Domain.FromIntervals([[0, 59]])
    ).OnlyEnforceIf(
        SP_isValid[i]
    )

# for i in range(N):
#     model.Add(
#         ( SP[i] + classes[i][0] > int(SP[i] / 6) * 6 + 6 ) 
#         if SP[i] != -1
#     )



for i in range(N):
    for j in range(i+1, N):
        if classes[i][1] == classes[j][1]:
            model.Add(
                SP[i] - SP[j] > classes[j][0]
            ).OnlyEnforceIf(
                SP[i] > -1
            ).OnlyEnforceIf(
                SP[j] > -1
            ).OnlyEnforceIf(
                SP[i] - SP[j] >= 0
            )

# for i in range(N):
#     model.Add(
#         (classes[i][2] <= room_capacities[R[i]])
#         if R[i] != -1 else True
#     )

# for i in range(N):
#     for j in range(i+1, N):
#         model.Add(
#             ( SP[i] > SP[j] + classes[j][0] )
#             or ( SP[i] + classes[i][0] < SP[j] )

#             if R[i] != -1 
#             and R[j] != -1
#             and R[i] == R[j]
#             else True
#         )

TypeError: 'BoundedLinearExpression' object is not iterable

In [35]:
for i in range(1, N+1):
        for j in range(1, N+1):
            if classes[i][1] == classes[j][1]:
                model.Add(
                    SP[i] - SP[j] >= classes[j][0]
                ).OnlyEnforceIf(
                    SP[i] >= SP[j]
                )
                # model.Add((SP[j] - SP[i] >= classes[i][0]) if SP[j]>=SP[i] else True)

TypeError: 'BoundedLinearExpression' object is not iterable

In [ ]:
# OBJECTIVES
model.Maximize(
    sum(map(lambda x: x >= 0, SP))
)

In [ ]:
solver = cp_model.CpSolver()
solver.Solve(model)